# Multilingual BERT Text-classifier

## Forberedelser

In [1]:
from data_representation import InputExample, InputFeatures, truncate_pairs
from transformers import AutoTokenizer, BertModel, BertForSequenceClassification
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
import torch
from torch import nn
import transformers as t
from sklearn import metrics 
import metrics
from tqdm import tqdm



import sys
sys.path.insert(1, "../lib")
from text_processer import *

import time
import random
import datetime
import re
from argparse import ArgumentParser

/usr/local/lib/python3.7/site-packages/spacy/util.py:271: UserWarning: [W031] Model 'nb_core_news_sm' (2.2.5) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.0). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Parametre

In [2]:
OLD_LABELS = False
DO_LOWER = False
TRAINING_SIZE = 0.8
MAX_LEN = 512
EPOCHS = 1
BATCHES = 1
LR = 1e-5 
EPS = 1e-8
RANDOM_SEED = 42
DEVICE = 'cpu'
PRED_TRESH = 0.5

In [3]:
"""
    Path to a *BERT* model only. Can either be a pretrained version or a finetuned version
"""
MODEL_PATH = "../hjemmesnekk/multilingual" # Path to BertModel


"""
    Path to a folder having both a finetuned BERT-version and trained dropout and classifier layers
"""
MODEL_PATH_TRAINED = "../hjemmesnekk/multilingual-trained/25.08_16_labels_8_epochs" # Path to BertModel


"""
    Path to folder where the trained model is going to be saved. Doublecheck for possible duplication
"""
SAVE_PATH = f"../hjemmesnekk/multilingual-trained/{datetime.date.today()}_{EPOCHS}_epochs"


"""
    Path to tokenizer, should not be changed
"""
TOKENIZER_PATH = "../hjemmesnekk/multilingual"


"""
    Path to config file, should not be changed
"""
CONFIG_PATH = "../hjemmesnekk/multilingual/config.json"

### Tokenizer

In [4]:
tokenizer = t.AutoTokenizer.from_pretrained(TOKENIZER_PATH, strip_accents = False, do_lower_case = DO_LOWER)

INFO:transformers.configuration_utils:loading configuration file ../hjemmesnekk/multilingual/config.json
INFO:transformers.configuration_utils:Model config BertConfig {
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "type_vocab_size": 2,
  "vocab_size": 119547
}

INFO:transformers.tokenization_utils_base:Model name '../hjemmesnekk/multilingual' not found in model shortcut name list (bert-base-uncased, bert-large-uncase

### Modellskall

In [5]:
class BertMultiLabler(nn.Module):
    """
        Class for multilabler BERT.
        Uses normal BertModel in bottom, a dropout layer and a linear classifier on top.
        
        If param loadTrained is True, the model loads all three layers.
        If param loadTrained is False, the model can either load a finetuned BERT layer with empty layers on top, 
        or just a pretrained BERT layer with empty layers on top. 

    """
    def __init__(self, loadTrained = True):
        super(BertMultiLabler, self).__init__()
        
        if loadTrained:
            print("Loading pretrained model, i.e. model where all three layers are finetuned")
            checkpoint = torch.load(MODEL_PATH_TRAINED)
            self.base_model = t.BertModel.from_pretrained(MODEL_PATH_TRAINED)
            self.droput = self.load_state_dict(checkpoint["dropout_state_dict"])
            self.classifier = self.load_state_dict(checkpoint["classifier_state_dict"])
            self.optimizer = self.load_state_dict(checkpoint["optimizer_state_dict"])
        else:
            print("Loading pretrained *BERT* model, but not pretrained classifier layer!")
            config = t.BertConfig.from_json_file(CONFIG_PATH)
            self.base_model = t.BertModel.from_pretrained(MODEL_PATH)
            self.dropout = nn.Dropout(config.hidden_dropout_prob)
            self.classifier = nn.Linear(config.hidden_size, NUM_LABELS)
            self.optimizer = torch.optim.Adam(params = self.parameters(), lr = LR)

    def forward(self, input_ids, masks, tokens):
        _, model_output = self.base_model(input_ids, attention_mask = masks, token_type_ids = tokens) # Model returns two params
        dropout_output = self.dropout(model_output)
        output = self.classifier(dropout_output)
        return output
    
    """
        Saving the BERT model and the other layers seperately
        Naive checker to assure that models is not overrided
    """
    def save_model(self, override = False):
        if not override and os.path.isdir(SAVE_PATH):
            SAVE_PATH += f"copy_{datetime.date.today()}/"
            
        model.base_model.save_pretrained(SAVE_PATH)
        torch.save({
            "dropout_state_dict":self.dropout.state_dict(),
            "classifier_state_dict":self.classifier.state_dict(),
            "optimizer_state_dict":self.optimizer.state_dict()
        }, SAVE_PATH)
        print("Model saved successfully at: ", SAVE_PATH)

### Tapsfunksjon

In [6]:
def loss_fn(outputs, targets):
    return nn.BCEWithLogitsLoss()(outputs, targets)

### Metoder for trening og validering

In [7]:
def train(epoch):
    model.train()
    for _,data in tqdm(enumerate(train_loader, 0)):
        ids = data['input_ids'].to(DEVICE, dtype = torch.long)
        mask = data['masks'].to(DEVICE, dtype = torch.long)
        token_type_ids = data['tokens'].to(DEVICE, dtype = torch.long)
        targets = data['labels'].to(DEVICE, dtype = torch.float)
        
        outputs = model(ids, mask, token_type_ids)
        model.optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%10==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        model.optimizer.zero_grad()
        loss.backward()
        model.optimizer.step()

### Oppbygging av data

In [8]:
class Custom_Data():
    """
        Takes in finished parsed data
        Creates instances to be used when training the model. 
    """
    def __init__(self, inputExamples, labels: list, tokenizer: t.tokenization_bert, MAX_LEN: int):
        self.inputExamples = inputExamples
        self.labels = labels
        self.tokenizer = tokenizer
        self.MAX_LEN = MAX_LEN
        self.label_map = {label : i for i,label in enumerate(labels)}
        
    def __len__(self):
        return len(self.inputExamples)
        
    """ 
        Method to tokenize one instance in inputExamples.
        Innebygget metode som tensordataset benytter seg av for å hente ut dataen
    """
    def __getitem__(self, ind):
        encoded_dict = self.tokenizer.encode_plus(self.inputExamples[ind].text_a,
                                            add_special_tokens = True,
                                            max_length = self.MAX_LEN,
                                            pad_to_max_length = True,
                                            return_attention_mask = True,
                                            truncation = True
                                            #return_tensors = 'pt',
        )
        input_ids = encoded_dict['input_ids']
        masks = encoded_dict['attention_mask']
        token_type_ids = encoded_dict['token_type_ids']
        
        labels = [0]*len(self.label_map.keys())
        for label in self.inputExamples[ind].labels:
            labels[self.label_map.get(label)] = 1            
            
        return {
            "text":self.inputExamples[ind].text_a, # Legger til text i dataloaders for uthenting senere
            "input_ids":torch.tensor(input_ids, dtype=torch.long),
            "masks":torch.tensor(masks, dtype = torch.long),
            "tokens":torch.tensor(token_type_ids, dtype=torch.long),
            "labels":torch.tensor(labels, dtype = torch.float)
        }  

### Metoder for uthenting av data

In [9]:
"""
    Returns the labeled_data in its entirety
"""
def load_labeled_data(with_intents = True) -> pd.DataFrame:
    if OLD_LABELS:
        l_df = pd.read_csv("../hjemmesnekk/labeled_data/toppoppgaver.csv").set_index("index")
        return l_df
    else:
        l_df = pd.read_csv("../hjemmesnekk/labeled_data/toppoppgaver_NYESTE.csv")
        l_df = metrics.convert_to_matrix(l_df).set_index("index")
        if not with_intents:
            l_df.drop(INTENTS, axis = 1)
        return l_df
    
"""
    Filters the labeled_data on prespecified columns
    Removes instances that are not labelled
    Removes df instances that are not in l_df
    Returns df and l_df
"""
def filter_labeled_data(df: pd.DataFrame, l_df: pd.DataFrame, cols: list = None):
    cols_to_use = cols if cols != None else l_df.columns.values
    l_df = l_df[cols_to_use]
    l_df["sum"] = l_df.sum(axis = 1)
    l_df = l_df[l_df["sum"] >= 1.0].drop("sum", axis = 1)
    
    df = df.loc[l_df.index.values,:]
    df.sort_index(axis = 0, inplace = True)
    l_df.sort_index(axis = 0, inplace = True)
    return df, l_df

def load_data(preprocessed = True) -> pd.DataFrame:
    t = text_processer(TOPPOPPGAVER)
    if preprocessed:
        return t.get_preprocessed_data()
    else:
        return t.get_cleaned_data()

def drop_cols(df, cols:list) -> pd.DataFrame:
    for col in df.columns.values:
        if col not in cols: 
            raise TypeError(f'Col: {col} not in param {cols}')
        assert col in cols
    return df.drop(cols, axis = 1)

"""
    Method to create inputExamples 
    Just instances with lemmatized texts and corresponding labels
"""
def createInputExamples(df, l_df, preprocessed = True):
    inputExamples = []   
    for i, val in enumerate(df["sentences"].values):
        labels = l_df.columns.values[np.where(l_df.values[i] == 1)]
        inputExamples.append(InputExample(i, val[0:MAX_LEN], None, labels))
    return inputExamples

### Metoder for parsing av data

In [10]:
# Denne må legges inn en JSON fil etterhvert - og helst kjøres ved preprosessering av data.
def lemmatize(x, do_lower = False):
    x = x.lower() if do_lower else x
    
    lemmas = {"vedrørende":r'(\bvedr[.]?)',
              "angående":r'(\bang.)',
              "angående":r'(\bang.)',
              "telefon":r'(\btlf[.]?)',
              "offentlig":r'(\boff.)',
              "konto":r'(\bkto.)',
              "kontonummer":r'(\bkontonr.)',
              "på grunn av ": r'(\bpga.)',
              "nummer":r'(\bnr[.]?)',
              "registrere":r'(\breg.)',
              "tidligere":r'(\btidl[.]?)',
              "med vennlig hilsen":r'(\bmvh[.]?)',
              "utbetaling": r'\butbet+al[ingenrt]*|(\butbet[.]?)',
              "dagpenger": r'\bdagpeng[aenr]{0,3}',
              "finne": r'(\bfinne[r]?)',
              "ferie": r'\bferie[ern]*',
              "arbeide": r'\barbeide[ret]{1,3}',
              "korona": r'(\bcorona[viruset]{0,6}\Z)|(\bkorona[viruset]{0,6}\Z)|(\bcovid[-19]{0,3}\Z)',
              "lege": r'(\blegen?)',
              "fastlege": r'(\bfastlegen)',
              "melding": r'(\bmld)|(\bmelding[enr]{0,3})',
              "lønn": r'(\blønn[nea]{0,3})',
              "pensjon": r'(\bpensjon[en]{0,2})|(\bpension[en]{0,2})',
              "informasjon": r'(\binformasjon[en]{0,2})|(\binfo[.]?)',
              "kontonummer": r'(\bkontonummer[et]{0,2})|(\bkontonr[.]?)',
              "oppdatere": r'(\boppdatert)|(\boppdatering)',
              "dokument": r'(\bdokument[ern]{0,3})|(\bdok)(|\bdoc)',
              "aktivitetsplan": r'(\bak?tivitets?plan[en]{0,3})',
              "endre": r'(\bendre)|(\bendring)',
              "ettersende": r'(\bettersende?)|({\bettersending})',
              "arbeidsledig":r'(\barb. ledig)|(\barb.ledig)',
              "arbeids":r'(\barb.)',
              "for eksempel":r'(\bf.eks.)|(\bf. eks.)',
              ". ":r'(\b\n)',
    }
    for key in lemmas.keys():
        x = re.sub(lemmas[key], key, x)
    return x

def lemmatize_data(df: pd.DataFrame, preprocessed = True) -> pd.DataFrame:
    pred = lambda x: lemmatize(x)
    df["sentences"] = df.raw_text.apply(pred) if preprocessed else df["Hva kom du hit for å gjøre"].apply(pred)
    return df

### Lager DataLoaders og deler opp i treningsdata og testdata

In [11]:
"""
    Takes in inputExamples
    Creates trainingset and testset by sampling from the inputExamples
    Returns trainingset and labelset
"""
def split_data(inputExamples: list):
    train_dataset = random.sample(inputExamples, int(TRAINING_SIZE*len(inputExamples)))
    test_dataset = [inputExample for inputExample in inputExamples if inputExample not in train_dataset]
    return train_dataset, test_dataset

def createDataLoaders(train_dataset, test_dataset, labels, tokenizer):
    training_set = Custom_Data(train_dataset, labels, tokenizer, MAX_LEN
    )
    
    test_set = Custom_Data(test_dataset,labels,tokenizer, MAX_LEN
    )
    
    train_loader = DataLoader(training_set, shuffle = True, batch_size = BATCHES)
    test_loader = DataLoader(test_set, sampler = SequentialSampler(test_set), batch_size = BATCHES)
    return train_loader, test_loader

## Kjører alt

#### Kolonner for treningsdata:

In [12]:
cols_to_use_old = ["Finne info/oversikt", "Ettersende dokumentasjon", "Permittert", "Sjekke/endre kontonr", "Finne skjema/dokumenter","Oppdatere/lage cv", "Pensjon", "Ukjent","Dagpenger"]
intent_cols = ["Trekke/Fjerne","Klage","Spørsmål","Finne","Registrere/lage","Endre/Oppdatere","Søke","Sjekke","Sende","@@@@@@@","Ikke oppgitt"]
cols_to_use = ["Annet","Pensjon","Dagpenger","Skjema/søknad","Utbetaling","Dokumentasjon","Tekniske problemer","Permittert","Personopplysning","SPAM","Informasjon","Skatt/Årsoppgave", "Arbeidssøker","Sykepenger","Sykemelding","CV"]
cols_to_use = None

In [13]:
df = load_data()
l_df = load_labeled_data()
df, l_df = filter_labeled_data(df, l_df, cols_to_use)
NUM_LABELS = len(l_df.columns.values)

#### Størrelse data:

In [14]:
print(l_df.sum(axis=0))

Saksbehandler/NAV      89
Spørsmål              149
Trekke/Fjerne          31
Skatt/Årsoppgave       90
Skjema/søknad         137
Kontakte               44
Registrere/lage       122
Klage                  80
Sjekke                269
Søke                  101
Personopplysning      140
Sykepenger             42
Sykemelding            69
Tekniske problemer    140
Barn/Foreldre          69
@@@@@@@                68
Utbetaling            130
SPAM                   70
Arbeidsgiver           64
Veileder               30
Meldekort              66
Sende                 158
Finne                 212
Permittert            203
Ikke oppgitt          879
Annet                 320
CV                     80
Arbeidssøker           78
Endre/Oppdatere       180
Ufør/AAP               43
Dokumentasjon         114
Dagpenger             157
Pensjon               197
Informasjon           154
Melding/brev           85
Vedtak/Sak             86
Annet språk            26
Aktivitetsplan         33
dtype: int64

#### Parser dataen og lager DataLoaders:

In [15]:
df = lemmatize_data(df, preprocessed = True)
inputExamples = createInputExamples(df, l_df, preprocessed = False)

In [16]:
train_data, test_data = split_data(inputExamples)
train_loader, test_loader = createDataLoaders(train_data, test_data, l_df.columns.values, tokenizer)

#### Initialiserer modellen:

In [17]:
model = BertMultiLabler(loadTrained = False)
model.to(DEVICE)

INFO:transformers.configuration_utils:loading configuration file ../hjemmesnekk/multilingual/config.json
INFO:transformers.configuration_utils:Model config BertConfig {
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "type_vocab_size": 2,
  "vocab_size": 119547
}

INFO:transformers.modeling_utils:loading weights file ../hjemmesnekk/multilingual/pytorch_model.bin


Loading pretrained *BERT* model, but not pretrained classifier layer!


INFO:transformers.modeling_utils:All model checkpoint weights were used when initializing BertModel.

INFO:transformers.modeling_utils:All the weights of BertModel were initialized from the model checkpoint at ../hjemmesnekk/multilingual.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use BertModel for predictions without further training.


BertMultiLabler(
  (base_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_af

#### Trener modellen

In [ ]:
for epoch in tqdm(range(EPOCHS)):
    train(epoch)

  0%|          | 0/8 [00:00<?, ?it/s]
0it [00:00, ?it/s]

Epoch: 0, Loss:  0.7036412358283997



1it [00:16, 16.26s/it]
2it [00:32, 16.24s/it]
3it [00:48, 16.22s/it]
4it [01:04, 16.13s/it]
5it [01:20, 16.05s/it]
6it [01:36, 15.94s/it]
7it [01:51, 15.93s/it]
8it [02:07, 15.92s/it]
9it [02:24, 16.01s/it]
10it [02:40, 16.10s/it]

Epoch: 0, Loss:  0.5543237328529358



11it [02:56, 16.15s/it]
12it [03:13, 16.27s/it]
13it [03:29, 16.22s/it]
14it [03:45, 16.09s/it]
15it [04:00, 15.89s/it]
16it [04:16, 15.87s/it]
17it [04:32, 15.88s/it]
18it [04:47, 15.76s/it]
19it [05:03, 15.63s/it]
20it [05:19, 15.78s/it]

...


71it [21:10, 11.16s/it]
72it [21:21, 11.15s/it]
73it [21:32, 11.25s/it]
74it [21:45, 11.55s/it]
75it [21:55, 11.28s/it]
76it [22:07, 11.26s/it]
77it [22:17, 10.96s/it]
78it [22:28, 11.04s/it]
79it [22:39, 10.97s/it]
80it [22:49, 10.79s/it]

Epoch: 7, Loss:  0.11241193115711212



81it [23:00, 10.88s/it]
82it [23:12, 11.25s/it]
83it [23:21, 10.49s/it]
84it [23:32, 10.48s/it]
85it [23:39,  9.68s/it]
86it [23:48,  9.51s/it]
87it [23:57,  9.10s/it]
88it [24:06,  9.31s/it]
89it [24:15,  8.93s/it]
90it [24:24,  9.11s/it]

Epoch: 7, Loss:  0.10598041117191315



91it [24:32,  8.72s/it]
92it [24:41,  8.92s/it]
93it [24:49,  8.70s/it]

### Lagrer modellen

In [25]:
model.save_model(override = True)

UnboundLocalError: local variable 'SAVE_PATH' referenced before assignment

#### Validerer modellen

## Kjører modellen på all dataen for å lage plots